In [ ]:
%load_ext aiida
%aiida

import urllib.parse as urlparse

import ase
import ipywidgets as ipw

# import pymol_render.render as pr
import matplotlib
import matplotlib.font_manager as font_manager
import matplotlib.patheffects as path_effects
import matplotlib.pyplot as plt
import numpy as np
from aiida import orm
from aiida_nanotech_empa.utils import cycle_tools as cyc
from aiidalab_widgets_base import viewer
from matplotlib.patches import Polygon, Rectangle

from surfaces_tools import helpers
from surfaces_tools.widgets import comments, obsolete

In [ ]:
pk = urlparse.parse_qs(urlparse.urlsplit(jupyter_notebook_url).query)["pk"][0]
workcalc = orm.load_node(pk)

In [ ]:
ase_atoms = workcalc.outputs.output_structure.get_ase()
ase_atoms_no_h_x = ase.Atoms(
    [a for a in ase_atoms if a.symbol != "H" and a.symbol != "X"]
)

In [ ]:
try:
    height = workcalc.inputs.height.value
except:
    height = 1.0

In [ ]:
cycles = cyc.dumb_cycle_detection(ase_atoms_no_h_x, 8)
centers, normals = cyc.find_cycle_centers_and_normals(ase_atoms_no_h_x, cycles, height)

In [ ]:
nics_atoms = [
    atom.index
    for atom in workcalc.outputs.output_structure.get_ase()
    if atom.symbol == "X"
]
sigma_arr = np.array(workcalc.outputs.output_parameters.dict.nmr_tensors)[
    nics_atoms, :, :
]
nics_zz_arr = []
nics_arr = []
for i_x in range(len(sigma_arr)):
    normal = normals[i_x]
    # normal = np.array([0.0, 0.0, 1.0])
    sigma = sigma_arr[i_x]
    nics_zz = -1.0 * np.dot(np.dot(normal, sigma), normal)
    # nics_zz = 0.5 * (nics_zz + -1.0*np.dot(np.dot(normal, sigma_2[i_x]), normal))
    nics_zz_arr.append(nics_zz)
    nics_arr.append(-1.0 * np.sum(np.diag(sigma)) / 3)

# NICS

In [ ]:
# average nics per cycle

# np.mean(nics_zz_arr)

In [ ]:
n_digits = 0

In [ ]:
ase_orient = ase_atoms
ase_orient_no_h_x = ase.Atoms(
    [a for a in ase_orient if a.symbol != "H" and a.symbol != "X"]
)

orient_cycles = cyc.dumb_cycle_detection(ase_orient_no_h_x, 8)
o_centers, o_normals = cyc.find_cycle_centers_and_normals(
    ase_orient_no_h_x, cycles, height
)

In [ ]:
# ase BBOX
x_min = np.min(ase_orient.positions[:, 0])
x_max = np.max(ase_orient.positions[:, 0])
y_min = np.min(ase_orient.positions[:, 1])
y_max = np.max(ase_orient.positions[:, 1])

In [ ]:
bond_i_arr, bond_j_arr = ase.neighborlist.neighbor_list("ij", ase_orient_no_h_x, 1.8)

In [ ]:
fig_x = (x_max - x_min + 2.0) / 2.5
fig_y = (y_max - y_min + 2.0) / 2.5

font = font_manager.FontProperties(fname="./miscellaneous/arial.ttf")

plt.figure(figsize=(fig_x, fig_y))
ax = plt.gca()
ax.set_aspect("equal")

# -------------------------------------------------------------------
# ATOMS
# for at in ase_atoms_no_h_x:
#    pos = at.position
#    circ = plt.Circle(pos[:2], 0.08, color='black', zorder=10)
#    ax.add_artist(circ)
# -------------------------------------------------------------------

# -------------------------------------------------------------------
# BONDS

# b_cmap_name = 'seismic_r'
# b_cmap_name = 'bwr_r'
# b_cmap = matplotlib.cm.get_cmap(b_cmap_name)
#
# b_max = benz_bond + 0.08
# b_min = benz_bond - 0.08

for bond_i, bond_j in zip(bond_i_arr, bond_j_arr):
    if bond_i < bond_j:
        p1 = ase_orient_no_h_x[bond_i].position
        p2 = ase_orient_no_h_x[bond_j].position
        dist = np.linalg.norm(p1 - p2)

        ax.plot(
            [p1[0], p2[0]],
            [p1[1], p2[1]],
            color="k",
            linewidth=1.0,
            solid_capstyle="round",
        )
        # ax.plot([p1[0], p2[0]], [p1[1], p2[1]], color=rgb, linewidth=3.0, solid_capstyle='round')


# -------------------------------------------------------------------
# NICS

cmap_name = "bwr_r"
# cmap_name = 'seismic_r'
cmap = matplotlib.colormaps.get_cmap(cmap_name)

cmap_min = -40
cmap_max = 40

for nics_val, cycl in zip(nics_zz_arr, orient_cycles):

    nics_norm = (nics_val - cmap_min) / (cmap_max - cmap_min)
    rgba = cmap(nics_norm)

    points = []
    for i_at in cycl:
        points.append(ase_orient_no_h_x[i_at].position[:2])

    polygon = Polygon(np.array(points), closed=True, color=rgba, zorder=1)
    ax.add_artist(polygon)

# -------------------------------------------------------------------

plt.axis("off")
plt.xlim([x_min, x_max])
plt.ylim([y_min, y_max])
# plt.savefig("./fig/%s.png" % label, dpi=400, bbox_inches='tight', transparent=True)
# plt.savefig("./fig/%s.pdf" % label, bbox_inches='tight', transparent=True)

# -------------------------------------------------------------------
# NICS text
for nics_val, cntr in zip(nics_zz_arr, o_centers):

    # circ = Circle((cntr[0], cntr[1]), radius=0.02, color='black', zorder=1)
    # ax.add_artist(circ)

    if n_digits == 0:
        val_txt = "%.0f" % int(np.round(nics_val))
    else:
        val_txt = ("%." + str(n_digits) + "f") % nics_val

    # txt = plt.text(cntr[0], cntr[1]-0.15, val_txt, horizontalalignment='center',
    #               verticalalignment='baseline', color='black', fontproperties=font,
    #               fontdict={'size': 24})
    #              #bbox=dict(color='red'))

    font_h = 0.35  # font size 24
    font_h = 0.28  # font size 20

    col = "black"
    if nics_val > 20.0 or nics_val < -50:
        col = "white"
    txt = plt.text(
        cntr[0],
        cntr[1] - font_h,
        val_txt,
        horizontalalignment="center",
        verticalalignment="baseline",
        color=col,
        fontdict={"size": 20},
    )
    # if nics_val > 20.0:
    #    txt.set_path_effects([path_effects.Stroke(linewidth=2, foreground='black'),
    #                   path_effects.Normal()])
    # bbox = txt.get_window_extent().inverse_transformed(plt.gca().transData)
    # print(bbox.y1 - bbox.y0)
    # txt.set_path_effects([path_effects.Stroke(linewidth=1.5, foreground='black'),
    #                   path_effects.Normal()])

# -------------------------------------------------------------------

# plt.savefig("./fig/%s_txt.png" % label, dpi=400, bbox_inches='tight', transparent=True)
# plt.savefig("./fig/%s_txt.pdf" % label, bbox_inches='tight', transparent=True)

plt.show()

# Comments

In [ ]:
comments_widget = comments.CommentsWidget(workchain=pk)
display(comments_widget)

# Mark calculation as obsolete 

In [ ]:
obsolete = obsolete.ObsoleteWidget(workchain=pk)
display(obsolete)